In [1]:
import matplotlib
!pip install transformers
!pip install datasets
VERSION = "1.8.1"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION


from datasets import load_dataset
from transformers import (AutoTokenizer, AutoConfig,
                          AutoModelForSequenceClassification, TrainingArguments, Trainer)

#from transformers import WarmupLinearSchedule as get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import torch


def load_dataset_new():
    file1 = open('/content/drive/My Drive/twitter-datasets/train_pos_full.txt', 'r')
    reviewspos = file1.readlines()
    file1 = open('/content/drive/My Drive/twitter-datasets/train_neg_full.txt', 'r')
    reviewsneg = file1.readlines()
    return reviewspos, reviewsneg


import re
def pre_process(text):
  text = re.sub(r"<user>", "",text)#remove <user>

  others = '1234567890@#+&*[]-%.:/();$=><|{}^' + "'`" + '_' #remove these symbols
  for p in others:
        text = text.replace(p,'')
  return text


In [2]:
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla
import torch_xla.core.xla_model as xm

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
reviewspos, reviewsneg = load_dataset_new()
reviews = reviewspos + reviewsneg
pos = [1] * 1250000
neg = [0] * 1250000
labels = pos + neg
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=.2)


In [6]:
import pandas as pd
dtrain = {'text': train_texts, 'labels': train_labels}
df = pd.DataFrame(data=dtrain)
df['text'] = df['text'].apply(lambda x: pre_process(x))
df.to_pickle("train_full.pkl")

In [7]:
dtest = {'text': val_texts, 'labels': val_labels}
df = pd.DataFrame(data=dtest)
df['text'] = df['text'].apply(lambda x: pre_process(x))
df.to_pickle("test_full.pkl")

In [8]:
datasetall = load_dataset("pandas", data_files={'train':"/content/train_full.pkl",'test':"/content/test_full.pkl" })


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-85b4822da8c7ff45/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
datasetall


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2000000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 500000
    })
})

In [10]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [11]:
# !pip install torch==1.10.0

In [14]:
choice='roberta-base'
tokenizer = AutoTokenizer.from_pretrained(choice, use_fast=True)

def tokenize_function(x):
      return tokenizer(x["text"], padding="max_length", truncation=True)
tokenized_datasets = datasetall.map(tokenize_function, batched=True)  
 
 
train_dataset = tokenized_datasets["train"].shuffle(seed=907).select(range(100000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=907).select(range(100000))

#modelconfig = AutoConfig.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-uncased', num_labels=2)
#model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', config=modelconfig)

#total_steps = len(dataloader) * 4
#scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps=0, t_total= total_steps)


  0%|          | 0/2000 [00:00<?, ?ba/s]

  0%|          | 0/500 [00:00<?, ?ba/s]

In [15]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #_, _, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        #'macro f1': macro_f1,
        'precision': precision,
        'recall': recall
    }

In [16]:
probs = [0.2]
rates = [8e-5]

for prob in probs:
  for rate in rates:


    modelconfig = AutoConfig.from_pretrained(pretrained_model_name_or_path=choice, num_labels=2,attention_probs_dropout_prob=prob )

    model = AutoModelForSequenceClassification.from_pretrained(choice, config=modelconfig)
    model.train()


    WRAPPED_MODEL = xmp.MpModelWrapper(model)

 

    #optimizer = torch.optim.Adam(model.parameters(),
    #              lr = 2e-5
    #             )
    def _mp_fn(index):
      device = xm.xla_device()
      # We wrap this 
      model = WRAPPED_MODEL.to(device)
      trainer = Trainer(
      model=model, args=TrainingArguments(output_dir = '/content/drive/MyDrive/twitter-datasets/modelssave', evaluation_strategy="epoch", learning_rate=rate),compute_metrics=compute_metrics, train_dataset=train_dataset, eval_dataset=eval_dataset)
      trainer.place_model_on_device = False

      trainer.train()

    xmp.spawn(_mp_fn, start_method="fork")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.404800,0.408758,0.836920,0.836683,0.839107,0.837016
2,0.347200,0.419362,0.857860,0.857663,0.859638,0.857776
3,0.324500,0.499262,0.860000,0.860000,0.860007,0.860006


Saving model checkpoint to /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-3500
Configuration saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-3500/config.json
Model weights saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-3500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-4000
Configuration saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-4000/config.json
Model weights saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-4500
Configuration saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-4500/config.json
Model weights saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-4500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `Rob

In [17]:
from huggingface_hub import notebook_login

notebook_login()

ERROR:root:HfApi.login: This method is deprecated in favor of `set_access_token`.


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [20]:
!sudo apt-get install git-lfs
!git lfs install --system --skip-repo

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.


In [22]:
model.push_to_hub("roberta-tiny-model-full",use_temp_dir=True )


/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:726: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/mollypak/roberta-tiny-model-full into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.38k/476M [00:00<?, ?B/s]

To https://huggingface.co/mollypak/roberta-tiny-model-full
   9f18822..174fd4f  main -> main

   9f18822..174fd4f  main -> main



'https://huggingface.co/mollypak/roberta-tiny-model-full/commit/174fd4f263975f64b67d3a87517d2700b9c4eb7a'

In [23]:
tokenizer.push_to_hub("roberta-tiny-model-full",use_temp_dir=True)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:726: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/mollypak/roberta-tiny-model-full into local empty directory.
To https://huggingface.co/mollypak/roberta-tiny-model-full
   174fd4f..4acdb92  main -> main

   174fd4f..4acdb92  main -> main



'https://huggingface.co/mollypak/roberta-tiny-model-full/commit/4acdb92d34f8fe8a8969bc111132f43456f26e7d'

In [ ]:

models = ['roberta-base','bert-base-uncased']

for choice in models:
    tokenizer = AutoTokenizer.from_pretrained(choice, use_fast=True)

    if choice=="gpt2":
      def tokenize_function(x):
        return tokenizer(x["text"])
      tokenized_datasets = datasetall.map(tokenize_function, batched=True)  

    if choice=="roberta-base":
      def tokenize_function(x):
        return tokenizer(x["text"], padding="max_length", truncation=True)
      tokenized_datasets = datasetall.map(tokenize_function, batched=True)  
 
    train_dataset = tokenized_datasets["train"].shuffle(seed=907).select(range(100000))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=907).select(range(100000))

    modelconfig = AutoConfig.from_pretrained(pretrained_model_name_or_path=choice, num_labels=2)

    model = AutoModelForSequenceClassification.from_pretrained(choice, config=modelconfig)
    model.train()


    WRAPPED_MODEL = xmp.MpModelWrapper(model)

 

    #optimizer = torch.optim.Adam(model.parameters(),
    #              lr = 2e-5
    #             )
    def _mp_fn(index):
      device = xm.xla_device()
      # We wrap this 
      model = WRAPPED_MODEL.to(device)
      trainer = Trainer(
      model=model, args=TrainingArguments(output_dir = '/content/drive/MyDrive/twitter-datasets/modelssave', evaluation_strategy="epoch"),compute_metrics=compute_metrics, train_dataset=train_dataset, eval_dataset=eval_dataset)
      trainer.place_model_on_device = False

      trainer.train()

    xmp.spawn(_mp_fn, start_method="fork")